# Homework 9 - Parameter Search and Feature Selection
In this assignment, you will be working with the cars dataset to perform parameter search and feature selection using various techniques. The goal is to explore the performance of a machine learning model by optimizing its parameters and selecting the most relevant features. We will try to predict the Make of the car based on its attributes.

Complete the missing parts in this guide.

### Step 1: Load Data
You can load the data from the provided CSV file using `pandas`.

### Step 2: Preprocess and split data
Use attributes `['Make', 'Engine HP', 'Engine Cylinders', 'Number of Doors', 'highway MPG', 'city mpg', 'MSRP']` (x) to predict the `Make` (y, target variable). Note that `Make` is a categorical variable, so you will need to use a classification model.
 - You should handle missing values appropriately (suggestion: drop rows with missing values).
 - You should remove Makes that have less than 20 samples.
 - You should split the data into training and testing sets.

### Step 3: Predict Make using a Decision Tree Classifier
Use a Decision Tree Classifier to predict the `Make` of the car. You will need to:
 - Train the model on the training set.
 - Evaluate the model on the test set using different metrics (use the `report` function from `sklearn.metrics`).
 - Showcase the confusion matrix to visualize the performance of the model. Do you see any interesting patterns in the confusion matrix?

### Step 4: Perform Parameter Search
Use `GridSearchCV` to perform a parameter search on the Decision Tree Classifier. You can use `criterion`, `max_depth`, and `splitter` as parameters to search over. Check the documentation for Decistion Tree Classifier to see the available parameters.
 - Discuss if the model performance improved after the parameter search.

### Step 5: Feature Selection
Use the best model from the parameter search to perform feature selection. Use `feature_importances_` to get the importance of each feature in the model. You can use `SelectFromModel` from `sklearn.feature_selection` to select the most important features based on the model's feature importances.
 - You can visualize the feature importances using a bar chart for better understanding.
 - Discuss which features are most important and how they impact the model's performance.
 
### Step 6: Plot ROC Curve and calculate AUC
Use the best model from the parameter search to plot the ROC curve and calculate the AUC (Area Under the Curve) score for one of the classes (e.g., `Toyota`). You can use `roc_curve` and `auc` from `sklearn.metrics` to do this. You can use the method `predict_proba` to get the probabilities for each class.

### Step 7: Repeat 3-6 for Logistic Regression, KNeighborsClassifier, RandomForestClassifier, and SVM(SVC).
 - Compare the performance of the classifiers using the same metrics as before.
 - Discuss which classifier performed best and why.
 - Check the documentation or ask for help if you are unsure about the parameters to use for each classifier. For instance, for KNeighborsClassifier, you can search over `n_neighbors` and `weights`.


## Dataset Overview
The cars dataset contains several attributes of various car models, including their specifications and performance metrics.

## Submission Guidelines

- Submit your completed notebook as a HTML export, or a PDF file.

To export to HTML, if you are on Jupyter, select `File` > `Export Notebook As` > `HTML`.

If you are on VSCode, you can use the `Jupyter: Export to HTML` command.
 - Open the command palette (Ctrl+Shift+P or Cmd+Shift+P on Mac).
     - Search for `Jupyter: Export to HTML`.
     - Save the HTML file to your computer and submit it via Canvas.

Make sure the plots appear in the exported file. If you are using plotly or more complicated interactive plots, make sure to a bitmap backend like 'png'.


Let's start by loading several libraries that we will need for this assignment.


In [ ]:
import pandas as pd
import sqlite3

from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.model_selection import cross_val_score, GridSearchCV, train_test_split
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

import matplotlib.pyplot as plt
import seaborn as sns

import numpy as np      # Added

Now we need to import the data. This time we will import the `carfeatures.csv` from the Datasets folder. Make any adjustments to the path as necessary.


In [ ]:
df = pd.read_csv("../Datasets/carfeatures.csv")
df.head()

In [ ]:
df.info()

We will use only a subset of the columns for this assignment. Let's select them:
We want to predict `Make`.

In [ ]:
# Our variables
x_features = [
'Engine HP',
'Engine Cylinders',
'Number of Doors',
'highway MPG',
'city mpg',
'MSRP'
]

# Target variable
y_feature = 'Make'


Next, we need to drop rows with missing values among the selected columns. (You can use `dropna()` method from pandas with a `subset` parameter that restricts the operation to the selected columns.)

In [ ]:
# remove rows with missing values in x_features

# Combine the feature columns and the target column
columns_to_check = x_features + [y_feature]

df = df.dropna(subset=columns_to_check) # COMPLETE

In [ ]:
df.info()

We need to remove rows where the `Make` has not many samples. We can use `value_counts()` to find the counts of each Make. I recommend to plot value counts to visualize the distribution of Makes. Let's see how many samples each `Make` has:

In [ ]:
# Let's see the distribution of the y_feature. We want to have classes with at least 20 samples.
categories_count = df[y_feature].value_counts() # COMPLETE
plt.figure(figsize=(10, 6))
categories_count.plot(kind='bar', title=f"{y_feature} Distribution")
#show horizontal line for 20 
plt.axhline(y=20, color='r', linestyle='--')
plt.xlabel(y_feature)
plt.ylabel('Count')
plt.show()

We need to remove any entries from `Make` that have less than 20 samples. One solution is getting the indices of the Makes that have less than 20 samples and then dropping those rows from the DataFrame. You can use `isin()` method to filter the DataFrame.

In [ ]:
# Remove classes with less than 20 samples
categories_to_remove = categories_count[categories_count < 20].index.tolist() # COMPLETE

print(categories_to_remove)

df = df[~df[y_feature].isin(categories_to_remove)] # COMPLETE



Define `X` variables (input) and the `y` variable (target) in terms of the features.

In [ ]:
X = df[x_features] # COMPLETE
y = df[y_feature] # COMPLETE

Let's now split the dataset into training and testing sets. Let's use a test_size of 0.3.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42) # COMPLETE


Let's train a simple DecisionTreeClassifier

In [ ]:
# Initialize classifiers
# Decision Tree first
dt_classifier = DecisionTreeClassifier() # COMPLETE
# Fit the model
dt_classifier.fit(X_train, y_train) # COMPLETE


Now let's predict and see the report for the decision tree classifier.

In [ ]:
# Predict on the test set
y_pred_dt = dt_classifier.predict(X_test) # COMPLETE
# Evaluate the Decision Tree model
print("Decision Tree Classifier:")
print(classification_report(y_test, y_pred_dt)) # COMPLETE


Plot the confusion matrix to visualize the performance of the model. You can use `confusion_matrix` from `sklearn.metrics` and `heatmap` from `seaborn` to visualize it.

In [ ]:

conf_matrix_dt = confusion_matrix(y_test, y_pred_dt) # COMPLETE

plt.figure(figsize=(12, 12))
sns.heatmap(conf_matrix_dt, annot=True, fmt='d', cmap='Blues', xticklabels=dt_classifier.classes_, yticklabels=dt_classifier.classes_)
plt.title('Confusion Matrix - Decision Tree Classifier')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()


Do you see any interesting patterns in the confusion matrix? Discuss briefly any observations you have.

**OBSERVATIONS**

- The dataset is imbalance... some makers/brand have more observations than others, like 
- The correct classifications (also known as true positives) are along the diagonal. 
- There is one case that looked to be frequently misclassified in both directions, GMC <-> Chevrolet.
- Chevrolet -> Buick and Buick -> Chevrolet looks to be symetric (same number of misclassifications).
- Pontiac <-> Chevrolet looks to another case with similar number of misclassifications in both directions.
- KIA <-> Hyundai, but notice that misclassify KIA as Hyundai happend more than Hyundai as KIA.
- Bentley and Ferrari look to be the high end car makers that are less likely to end up misclassified.

---

Now let's explore the space of parameters for the Decision Tree Classifier. We will use `GridSearchCV` to perform a parameter search on the Decision Tree Classifier. We will search over `criterion`, `max_depth`, and `splitter`.

In [ ]:
# Parameter space for Decision Tree Classifier
# Grid search will evaluate all combinations of these parameters
desicion_tree_params_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [4, 5, 6, 7, 8, 9, 10, 11, 12, 15, 20, 30, 40, 50],
    'splitter': ["best", "random"]
}

# Perform grid search with cross-validation for Decision Tree Classifier
grid_search_decision_tree_classifier = GridSearchCV(
    DecisionTreeClassifier(),
    desicion_tree_params_grid,
    cv=10, # 10-fold cross-validation
    scoring='f1_macro', # Use F1 score for multiclass classification
    verbose=1
    )

grid_search_decision_tree_classifier.fit(X_train, y_train)

print("Decision Tree best grid score in cv: " + str(grid_search_decision_tree_classifier.best_score_))
print("Decision Tree grid test score: " + str(grid_search_decision_tree_classifier.score(X_test, y_test)))

# DEBUGGING
print("\nDecision Tree grid training score: " + str(grid_search_decision_tree_classifier.score(X_train, y_train)))


We can check the best parameters found by the grid search and the best score achieved.

In [ ]:
decision_tree_best_params = grid_search_decision_tree_classifier.best_params_
print("Decision Tree best params: " + str(decision_tree_best_params))

The `grid_search_decision_tree_classifier` object works as a model. You can fit and transform data using it. Let's fit the model to the training data and then evaluate it on the test set.
So let's see how the model performs after the parameter search. We will use the `report` function to evaluate the model on the test set.

In [ ]:
decision_tree_best_params = grid_search_decision_tree_classifier.best_params_
print("Decision Tree best params: " + str(decision_tree_best_params))

Now we can run `predict()` on our `grid_search_decision_tree_classifier`.

In [ ]:
y_pred = grid_search_decision_tree_classifier.predict(X_test)

Let's look at the resulting report. Call `classification_report()` below.

In [ ]:
grid_search_decision_tree_classification_report = classification_report(y_test, y_pred)
print("Decision Tree Classification report with test data")
print(grid_search_decision_tree_classification_report)

Now, let's see the feature importances of the Decision Tree Classifier. You can access the `feature_importances_` attribute of the trained model.

In [ ]:
# Feature importance
feature_importances_dt = dt_classifier.feature_importances_ # COMPLETE
# Plot feature importances (you can use matplotlib or seaborn)


# YOUR CODE HERE

feature_importance_df = pd.DataFrame({'Feature': x_features, 'Importance': feature_importances_dt})

plt.figure(figsize=(10, 6))
sns.barplot(x='Importance', y='Feature', hue='Feature', data=feature_importance_df, palette='viridis', legend=False)
plt.title('Feature Importance from Decision Tree Classifier')
plt.xlabel('Feature Importance')
plt.ylabel('Feature')
plt.tight_layout()
plt.show()



We'll need to grab the features to use now, using the `SelectFromModel()` function. Then, let's run `fit()` on `select`.

In [ ]:
select = SelectFromModel(DecisionTreeClassifier(), threshold='median')

# which features were selected?
print("Features before selection:")
print(X_train.columns.tolist())

select.fit(X_train, y_train) # Your Code Here
X_train_selected = select.transform(X_train)
X_test_selected = select.transform(X_test)

# which features were selected?
selected_features = X_train.columns[select.get_support()]
print("\nSelected features after selection:")
print(selected_features.tolist())

Now let's apply those best params froem the grid search. Assign the respective fields from `decision_tree_best_params` for your classifier.

In [ ]:
# Applying DecisionTreeClassifier using the best params from the grid search and with selected data
decision_tree_classifier = DecisionTreeClassifier(**decision_tree_best_params)

We need to run the `fit()` function using `X_train_selected` and `y_train` as parameters. Then, run `predict()` using `X_test_selected`.

In [ ]:
decision_tree_classifier.fit(X_train_selected, y_train) # COMPLETE
y_pred_selected = decision_tree_classifier.predict(X_test_selected) # COMPLETE

In [ ]:
# Get training accuracy DEBUGGING
train_accuracy = decision_tree_classifier.score(X_train_selected, y_train)
print(f"Training Accuracy: {train_accuracy}")

In [ ]:
# Calculate the correlation matrix
corr_matrix = X.corr()

# Display the correlation matrix as a heatmap
plt.figure(figsize=(6, 4))
sns.heatmap(corr_matrix, annot=True, fmt=".2f", cmap='coolwarm')
plt.title('Correlation Matrix of Features')
plt.show()

Lastly, rerun the `classification_report()` and print out what your results are.

In [ ]:
# Lastly, rerun the `classification_report()` and print out what your results are.
decision_tree_classification_report_selected = classification_report(y_test, y_pred_selected) # COMPLETE
print("Decision Tree Classification report with selected features")
print(decision_tree_classification_report_selected)


**Do you see any improvements?**

Not any major improvement. The accuracy of the model decrease from 0.81 → 0.78 is small, about ~3%.

So it could suggest:

- The features dropped (`Engine Cylinders`, `Number of Doors`, `city mpg`) do not look to be critically important.

- A smaller feature set (variables that explain) implies a simpler, faster, more general, and easier model to interpret.

- Notice that there features that have high correlation, e.g. `city mpg` and `highway MPG` so by dropping `highway MPG` so we end up with less multicollinearity, hence improving model robustness, and trading-off a small additional information that the extra feature may bring (high correlation between features mean that we have redundant information in a way). A similar case is observed between `Engine HP` and `Engine Cylinders`.

Looking at the training accuracy, it dropped a bit (0.99 to 0.98) so a bit of over-fitting went away - but nothing major, I will think. 

---

Finally, let's plot the ROC (Receiver Operating Characteristic) curve to evaluate the performance of our model. Note that we will need to use the `predict_proba()` method to get the probabilities for each class. Feel free to choose one of the classes, for example, `Toyota`. Note that Decision Tree Classifier is not the best for ROC curves as they are not probabilistic models, but we can still plot it for the sake of this exercise. The ROC curves for the other classifiers will be more meaningful.

Plot the ROC with a line for the random classifier (diagonal line) and the ROC curve for the Decision Tree Classifier. Calculate the area under the curve (AUC). You can use `roc_curve` and `auc` from `sklearn.metrics`.

In [ ]:
# ROC
from sklearn.metrics import roc_curve, auc
y_score = grid_search_decision_tree_classifier.predict_proba(X_test)
chosenClass = "Toyota"
chosenClassIndex = grid_search_decision_tree_classifier.classes_.tolist().index(chosenClass)
fpr, tpr, _ = roc_curve(y_test == chosenClass, y_score[:, chosenClassIndex])
roc_auc = auc(fpr, tpr)


# Plot ROC curves: tpr vs fpr (x-axis is fpr, y-axis is tpr)
# Don't forget to draw the random guess line (y=x) use dash line 

# YOUR CODE HERE
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='blue', lw=2, label=f'Decision Tree (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='gray', lw=2, linestyle='--', label='Random Classifier')

plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate (FPR)')
plt.ylabel('True Positive Rate (TPR)')
plt.title(f'ROC Curve - Decision Tree Classifier for "{chosenClass}"')
plt.legend(loc="lower right")
plt.grid(True)
plt.show()

### Now let's repeat the steps 3-6 for Logistic Regression, KNeighborsClassifier, RandomForestClassifier, and SVM (SVC).

For Logistic Regression and SVM we recommend using standardized X features. You can use `StandardScaler` from `sklearn.preprocessing` to standardize the features.

For instance, let's prepare the data for Logistic Regression and SVC:

In [ ]:
# Logistic regression works best if the data is scaled, so we will scale the data using StandardScaler.
from sklearn import preprocessing

scaler = preprocessing.StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)


Now similarly perform the above operations for Logistic Regression

In [ ]:
# Your Code Here

lr= LogisticRegression(C=1000.0, random_state=42,max_iter=1000) #Your Code Here
# Using settings from HW 7

lr.fit(X_train_scaled, y_train.to_numpy())  # .ravel() will be deprecated
y_pred= lr.predict(X_test_scaled)

print(f'Accuracy Logistic Regression: {accuracy_score(y_test, y_pred)}')

In [ ]:
decision_tree_classification_report = classification_report(y_test, y_pred, zero_division=np.nan)
print("Logistic Regression Classification report")
print(decision_tree_classification_report)

Now similarly perform the above operations for KNeighborsClassifier. 

In [ ]:
# Your Code Here

from sklearn.neighbors import KNeighborsClassifier

# Using one neighbor 
knn = KNeighborsClassifier(n_neighbors=1, metric='euclidean') # Your Code here

#Your Code to fit the model here
knn.fit(X_train_scaled, y_train)

y_pred_knn = knn.predict(X_test_scaled) # Your Prediction Code Here

# Your Accuracy Output Code Here
print(f'Accuracy KNN: {accuracy_score(y_test, y_pred_knn)}')

In [ ]:
knn_decision_tree_classification_report = classification_report(y_test, y_pred_knn, zero_division=np.nan)
print("KNN Classification report")
print(knn_decision_tree_classification_report)

Now similarly perform the above operations for Random Forest. 

In [ ]:
# Your Code Here

# Using some default for random forest

model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train_scaled, y_train)
y_pred_rf = model.predict(X_test_scaled)
print(f'Accuracy Random Forest: {accuracy_score(y_test, y_pred_rf):.4f}')


In [ ]:
RF_decision_tree_classification_report = classification_report(y_test, y_pred_rf, zero_division=np.nan)
print("Random Forest Classification report")
print(RF_decision_tree_classification_report)

Now similarly perform the above operations for SVM (SVC).

In [ ]:
# Your Code Here
from sklearn.svm import SVC

# Using RBF as it looks to work better than linear kernel

svm = SVC(kernel='rbf') # Your Code Here
    # kernel='linear', C=1.0, random_state=0, cache_size=7000

svm.fit(X_train_scaled, y_train.to_numpy())     # ravel() will be deprecated
y_pred_svm = svm.predict(X_test_scaled) # Your Prediction Code Here

print(f'Accuracy Support Vector Machine with a Radial Basis Function (RBF) kernel: {accuracy_score(y_test, y_pred_svm)}')

In [ ]:
svm.get_params()

In [ ]:
svm_decision_tree_classification_report = classification_report(y_test, y_pred_svm, zero_division=np.nan)
print("SVM Classification report")
print(svm_decision_tree_classification_report)

Based on all the classification reports, which classifier performed best? Discuss the performance of each classifier and the impact of feature selection and parameter tuning on the results.

**YOUR DISCUSSION HERE**

#### **Summary Table** ####

|Accuracy| Result|
|---|:---:|
|Logistic Regression|0.20770756588268632
|KNN| 0.8231793709266081
|Random Forest| 0.8461
|Support Vector Machine with<br/> a Radial Basis Function (RBF) kernel| 0.30036837631056956

**Random Forest** is a powerful ensemble method that typically handles noise, irrelevant features, and non-linear data very well. It benefits from aggregating many decision trees, which reduces overfitting and improves generalization, likely explaining its better performance in comparison to the other models.

<br/>

**KNN** is a simple yet effective method when the data has clear class boundaries. Its good performance suggests that the features selected offer good clustering/separation between classes. Also, the Scaling was a factor that benefited KNN model.

Interesting to observe that there are some makers where KNN is outperforming Random Forest (Aston Martin is the more evident), like: 

|Make|Random Forest Classification|||KNN Classification|||
|---|:---:|:---:|:---:|:---:|:---:|:---:|
|  |precision (RF)|	recall (RF)|	f1-score (RF)|	precision (KNN)|	recall (KNN)|	f1-score (KNN)|
|Aston Martin|0.79|	0.95|	0.86|	1|	1|	1|
|Ferrari   | 0.96|	1|	0.98|	1|	1|	1|
|Land Rover| 0.92|	0.9|	0.91|	0.95|	0.92|	0.94|
|Lexus     | 0.76|0.85|	0.81|	0.81|	0.84|	0.82|
|Lotus     | 0.9|1|	0.95|	1|	1|	1|
|Maserati  | 1  |0.91	|0.95	|0.96	|0.96	|0.96|
|Pontiac   |0.68|	0.57|	0.62|	0.7|	0.62|	0.66|


For the following makers, the have pretty much the same result.

|Make|Random Forest Classification|||KNN Classification|||
|---|:---:|:---:|:---:|:---:|:---:|:---:|
|  |precision (RF)|	recall (RF)|	f1-score (RF)|	precision (KNN)|	recall (KNN)|	f1-score (KNN)|
|Cadillac|	0.9|	0.94|	0.92|	0.91|	0.92|	0.92|
|FIAT	|0.89	|0.94	|0.92	|0.89	|0.94	|0.92|
|Lamborghini|	0.94	|1	|0.97	|0.94	|1	|0.97|
|Rolls-Royce|	1	|0.82	|0.9	|1	|0.82	|0.9|

<br/>

**SVM with RBF Kernel** usually does well on non-linear problems, but its poor performance suggests that the parameters (like C, gamma) were not well tuned.

If we play with the `C` and `gamma` parameters then we can find a better model:

|Tuned Support Vector Machine|Accuracy |
|---|---|
|Using Radial Basis Function (RBF) kernel<br/>using `C` = 10, `gamma` = 1|0.6225559648625673|

<br/>

**Logistic Regression** works best when the decision boundary between classes is linear. Its low accuracy suggests that the data has complex, non-linear relationships that Logistic Regression couldn't capture. The Scaling helped the model but not enough. In comparison, the other models looked to be handling better the non-linearity.

Logistic Regression shows recall zero for 8 Makers (Acura,FIAT,Hyundai,Infiniti,Kia,Lincoln,Mitsubishi,Scion), it fails to identify any of the positive instances for any of these makers.

<br/>

**Final Remarks**

Considering how simple KNN works and it can be explained then it it is remarkable that it accomplished in comparison to the best one (Random Forest).

In [ ]:
# Try different parameters for SVM 

for parameter_C in [1, 10]:
    for parameter_gamma in [1, 0.1]:

        svm = SVC(kernel='rbf', C=parameter_C, gamma=parameter_gamma) # Your Code Here

        svm.fit(X_train_scaled, y_train.to_numpy())     # ravel() will be deprecated
        y_pred_svm = svm.predict(X_test_scaled) # Your Prediction Code Here

        print(f'\nParameter C: {parameter_C} | Parameter Gamma: {parameter_gamma}')
        print(f'Accuracy Support Vector Machine with a Radial Basis Function (RBF) kernel: {accuracy_score(y_test, y_pred_svm)}')


In [ ]:
# Using better settings for SVM

parameter_C = 10
parameter_gamma = 1

svm = SVC(kernel='rbf', C=parameter_C, gamma=parameter_gamma) # Your Code Here

svm.fit(X_train_scaled, y_train.to_numpy())     # ravel() will be deprecated
y_pred_svm = svm.predict(X_test_scaled) # Your Prediction Code Here

print(f'\nParameter C: {parameter_C} | Parameter Gamma: {parameter_gamma}')
print(f'Accuracy Support Vector Machine with a Radial Basis Function (RBF) kernel: {accuracy_score(y_test, y_pred_svm)}')


In [ ]:
tunned_svm_decision_tree_classification_report = classification_report(y_test, y_pred_svm, zero_division=np.nan)
print("SVM Classification report (gamma=1, C=10)")
print(tunned_svm_decision_tree_classification_report)